Sentiment

In [12]:
import transformers, logging
logging.basicConfig(level=logging.INFO)
transformers.logging.get_verbosity = lambda: logging.NOTSET 
import torch, tensorflow
from transformers import pipeline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import scipy
import tqdm
from tqdm import tqdm

PubMed

In [ ]:
!pip install pymed

In [16]:
from pymed import PubMed
pubmed  = PubMed()
query = 'cancer research'
results = pubmed.query(query, max_results=100)

article_list = []
for article in tqdm(results):
    article_dict = article.toDict()
    article_list.append(article_dict)
df= pd.DataFrame(article_list)
df.head()[:1]

100it [00:01, 71.65it/s]


,pubmed_id,title,abstract,keywords,journal,publication_date,authors,methods,conclusions,results,copyrights,doi,xml
0,40013415,A Randomized Double-Blind Controlled Evaluatio...,Dry and chapped lips adversely affect both lip...,"[aging, antioxidant, cheilitis, herbal lip hyd...",Journal of cosmetic dermatology,2025-02-27,"[{'lastname': 'Sutthiboonyapan', 'firstname': ...",None,The study findings support the therapeutic pot...,The herbal extracts demonstrated potential in ...,© 2025 The Author(s). Journal of Cosmetic Derm...,10.1111/jocd.70041,"[[[], [<Element 'Year' at 0x7cdd709d1b20>, <El..."


In [17]:
df.columns

Index(['pubmed_id', 'title', 'abstract', 'keywords', 'journal',
       'publication_date', 'authors', 'methods', 'conclusions', 'results',
       'copyrights', 'doi', 'xml'],
      dtype='object')

In [18]:
df = df[['pubmed_id', 'title', 'abstract', 'keywords', 'journal', 'publication_date']]
df= df[['title', 'abstract']].dropna().reset_index(drop=True)
df.head()[:1]

,title,abstract
0,A Randomized Double-Blind Controlled Evaluatio...,Dry and chapped lips adversely affect both lip...


Sentence Transformers

In [19]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

In [55]:
DB = ["Investigations on the clonal nature of antibody formation have demonstrated the expression of allelic exclusion by lymphoid cells (1), as well as the production of a monospecific, restricted population of antibody molecules by the clonal progeny of a single antibody-forming precursor cell (B cell) (2, 3).1 "
"Recent studies suggest, however, that a single clone may synthesize antibody molecules of the same speci- ficity which differ in heavy chain class. ",
"Thus, idiotypic determinants, which are con- sidered to be a function of the antibody combining site and therefore a variable re- gion marker (4), have been shown to be shared among the IgM and IgG anti-Sal- monella antibodies produced by individual rabbits (5). ",
]

In [56]:
DB_embeddings = model.encode(DB, convert_to_tensor=True)

In [57]:
query = "monoclonal antibody"
query_embedding = model.encode(query, convert_to_tensor=True)
distances = util.pytorch_cos_sim(query_embedding, DB_embeddings)[0]
distances

tensor([0.6020, 0.4960, 0.2870])

Sentence Transformers Search Engine

In [58]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [59]:
database = df['abstract'].values
database_embeddings = model.encode(database,convert_to_tensor=True)

In [60]:
def question(query, top_k):
  print(f'_______ {query} ______________')
  query_embedding = model.encode(query, convert_to_tensor=True)
  distances_cos_scores = util.pytorch_cos_sim(query_embedding, database_embeddings)[0]
  top_results = torch.topk(distances_cos_scores, k=top_k)
  for score, idx in zip(top_results[0], top_results[1]):
    print(f'Score {score:.4f} \t Idx {database[idx]}')
    print(f'Title: {df.loc[float(idx)].title}')
    print(f'Abstract: {df.loc[float(idx)].abstract}')

In [61]:
query = "monoclonal antibody"
question(query, 5)

_______ monoclonal antibody ______________
Score 0.4556 	 Idx Monoclonal antibody therapy using CD38 as a target remains central to managing human multiple myeloma (MM). CD38 was selected early on as a target for mAb-mediated therapy for MM, driven by findings from an early Cluster of Differentiation (CD) Workshop. The first CD38-targeting antibody to be approved yielded strong trial results, significantly improving survival rates and earning widespread patient acceptance. However, resistance to the therapy later emerged, complicating treatment management. Despite CD38's still central role in MM therapy, too little attention has been paid to its broader roles-not only as a myeloma marker but also as an enzyme and adhesion molecule in physiology. This review, a collaborative effort between basic scientists and clinical experts, explores some of the lesser-known mechanisms of antibody action and interactions with CD38 at key stages of treatment. The review also highlights the relevance o